# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
from psycopg2 import Error
import pandas as pd
from sql_queries import *

In [2]:
import create_tables
create_tables.main()
try:
    conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
    cur  = conn.cursor()
except Error as e:
    print("COULD NOT CONNECT TO DATABASE:")
    print(e)

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files("data/song_data")

In [5]:
filepath = song_files[0]

In [6]:
df = pd.read_json(filepath, lines = True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0].tolist()
print(song_data)

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
print(song_table_insert)
print(song_data)
try:
    cur.execute(song_table_insert, song_data)
    conn.commit()
    print("SUCCESFUL IMPORT")
except Error as e:
    print("ERROR IMPORTING SONG:")
    print(e)


INSERT INTO songs 
    (song_id, title, artist_id, year, duration) 
        VALUES (%s, %s, %s, %s, %s)
    ON CONFLICT (song_id) 
        DO NOTHING;

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]
SUCCESFUL IMPORT


Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
artist_data = df[['artist_id', 'artist_name', 'artist_location','artist_latitude', 'artist_longitude']].values[0].tolist()
print(artist_data)

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan]


#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
print(artist_table_insert)
print(artist_data)
try:
    cur.execute(artist_table_insert, artist_data)
    conn.commit()
    print('SUCCESSFUL INSERT')
except Error as e:
    print("ERROR IMPORTING ARTIST:")
    print(e)


INSERT INTO artists
    (artist_id, artist_name, artist_location, artist_latitude, artist_longitude) 
        VALUES (%s, %s, %s, %s, %s)
    ON CONFLICT (artist_id)
        DO NOTHING;

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan]
SUCCESSFUL INSERT


Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [11]:
log_files = get_files("data/log_data/")

In [12]:
filepath = log_files[0]

In [13]:
df = pd.read_json(filepath, lines = True) 
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Won't Go Home Without You,200,1543540625796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Train,Logged In,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Hey_ Soul Sister,200,1543540856796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [14]:
df = df.loc[df['page'] == 'NextSong']
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Won't Go Home Without You,200,1543540625796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Train,Logged In,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Hey_ Soul Sister,200,1543540856796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


In [15]:
t = pd.to_datetime(df['ts'], unit='ms')
t.head()

0   2018-11-30 00:22:07.796
1   2018-11-30 01:08:41.796
2   2018-11-30 01:12:48.796
3   2018-11-30 01:17:05.796
4   2018-11-30 01:20:56.796
Name: ts, dtype: datetime64[ns]

In [16]:
time_data = [df.ts.values, 
             t.dt.hour.values, 
             t.dt.day.values,
             t.dt.weekofyear.values, 
             t.dt.month.values, 
             t.dt.year.values,
             t.dt.weekday.values]
column_labels = ["start_time", "hour", "day", "week", "month", "year", "weekday"]

In [17]:
time_df = pd.DataFrame(dict(zip(column_labels, time_data)))
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,1543537327796,0,30,48,11,2018,4
1,1543540121796,1,30,48,11,2018,4
2,1543540368796,1,30,48,11,2018,4
3,1543540625796,1,30,48,11,2018,4
4,1543540856796,1,30,48,11,2018,4


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [18]:
print(time_table_insert)
print(time_df.head())
for i, row in time_df.iterrows():
    try:
        cur.execute(time_table_insert, list(row))
        conn.commit()
    except Error as e:
        print(e)


INSERT INTO time
    (start_time, hour, day, week, month, year, weekday) 
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    ON CONFLICT (start_time)
        DO NOTHING;

      start_time  hour  day  week  month  year  weekday
0  1543537327796     0   30    48     11  2018        4
1  1543540121796     1   30    48     11  2018        4
2  1543540368796     1   30    48     11  2018        4
3  1543540625796     1   30    48     11  2018        4
4  1543540856796     1   30    48     11  2018        4


Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [19]:
user_df = df[["userId", "firstName","lastName", "level", "gender", "location"]]
print(user_df.values[0].tolist())

['91', 'Jayden', 'Bell', 'free', 'M', 'Dallas-Fort Worth-Arlington, TX']


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [20]:
print(user_table_insert)
print(user_df.head())
for i, row in user_df.iterrows():
    try:
        cur.execute(user_table_insert, row)
        conn.commit()
    except Error as e:
        print(e)


INSERT INTO users
    (user_id, first_name, last_name, level, gender, location)
        VALUES (%s, %s, %s, %s, %s, %s)
    ON CONFLICT (user_id)
        DO NOTHING;

  userId firstName lastName level gender                             location
0     91    Jayden     Bell  free      M      Dallas-Fort Worth-Arlington, TX
1     73     Jacob    Klein  paid      M  Tampa-St. Petersburg-Clearwater, FL
2     73     Jacob    Klein  paid      M  Tampa-St. Petersburg-Clearwater, FL
3     73     Jacob    Klein  paid      M  Tampa-St. Petersburg-Clearwater, FL
4     73     Jacob    Klein  paid      M  Tampa-St. Petersburg-Clearwater, FL


Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [21]:
print(songplay_table_insert)
print(song_select)
for index, row in df.iterrows():
    # get songid and artistid from song and artist tables
    try: 
        cur.execute(song_select, (row.song, row.artist, row.length))
        results = cur.fetchone()
    except Error as e:
        results = None
        print(e)
    
    #
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
    
    # insert songplay record
    songplay_data = (row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    
    #
    try:
        cur.execute(songplay_table_insert, songplay_data)
        conn.commit()
    except Error as e:
        print(e)



INSERT INTO songplays
    (timestamp, user_id, level, song_id, 
    artist_id, session_id, location, user_agent)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    ON CONFLICT 
        DO NOTHING


SELECT songs.song_id, artists.artist_id 
    FROM songs 
        JOIN artists 
        ON songs.artist_id = artists.artist_id 
    WHERE songs.title = %s 
        AND artists.artist_name = %s 
        AND songs.duration = %s



Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [22]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.

In [23]:
import etl
etl.main()

71 files found in data/song_data
/home/workspace/data/song_data/A/A/A/TRAAAAW128F429D538.json
1/71 files processed.
/home/workspace/data/song_data/A/A/A/TRAAAVG12903CFA543.json
2/71 files processed.
/home/workspace/data/song_data/A/A/A/TRAAARJ128F9320760.json
3/71 files processed.
/home/workspace/data/song_data/A/A/A/TRAAAPK128E0786D96.json
4/71 files processed.
/home/workspace/data/song_data/A/A/A/TRAAABD128F429CF47.json
5/71 files processed.
/home/workspace/data/song_data/A/A/A/TRAAAMO128F1481E7F.json
6/71 files processed.
/home/workspace/data/song_data/A/A/A/TRAAAMQ128F1460CD3.json
7/71 files processed.
/home/workspace/data/song_data/A/A/A/TRAAAVO128F93133D4.json
8/71 files processed.
/home/workspace/data/song_data/A/A/A/TRAAAEF128F4273421.json
9/71 files processed.
/home/workspace/data/song_data/A/A/A/TRAAAFD128F92F423A.json
10/71 files processed.
/home/workspace/data/song_data/A/A/A/TRAAADZ128F9348C2E.json
11/71 files processed.
/home/workspace/data/song_data/A/A/B/TRAABNV128F425C